# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

Agent includes AgentRunner and AgentWorker

AgentRunner Orchestrator: mapping from task-id to taskstate. TaskState includes:
- Task
- Completed steps
- Step queue
Memorry: Conversation memory

AgentWorker: task reasoning and execution:
Tools: 
- Vector tool
- Summary tool

In [4]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, SummaryIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters, FilterCondition
from typing import List, Optional


def get_doc_tools(
        file_path: str,
        name: str,
) -> str:
    """Get vector query and summary query tools from a document."""

    # load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()
    splitter = SentenceSplitter(chunk_size=1024)
    nodes = splitter.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

    def vector_query(
            query: str,
            page_numbers: Optional[List[str]] = None
    ) -> str:
        """Use to answer questions over the MetaGPT paper.

        Useful if you have specific questions over the MetaGPT paper.
        Always leave page_numbers as None UNLESS there is a specific page you want to search for.

        Args:
            query (str): the string query to be embedded.
            page_numbers (Optional[List[str]]): Filter by set of pages. Leave as NONE
                if we want to perform a vector search
                over all pages. Otherwise, filter by the set of specified pages.

        """

        page_numbers = page_numbers or []
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]

        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            )
        )
        response = query_engine.query(query)
        return response

    vector_query_tool = FunctionTool.from_defaults(
        name=f"vector_tool_{name}",
        fn=vector_query
    )

    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
    )
    summary_tool = QueryEngineTool.from_defaults(
        name=f"summary_tool_{name}",
        query_engine=summary_query_engine,
        description=(
            "Use ONLY IF you want to get a holistic summary of MetaGPT. "
            "Do NOT use if you have specific questions over MetaGPT."
        ),
    )

    return vector_query_tool, summary_tool

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [7]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
Agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and creating a detailed PRD. The Architect translates requirements into system design components. The Project Manager handles task distribution, while Engineers execute designated classes and functions. The QA Engineer formulates test cases to ensure code quality. Each role has specific responsibilities and expertise tailored to different aspects of the collaborative software development process within the MetaGPT framework.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "communication between agent roles in MetaGPT"}
=== Function O

In [8]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-07-13
last_modified_date: 2024-07-13

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [9]:
response = agent.chat("Tell me about the evaluation datasets used.")

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and SoftwareDev.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and SoftwareDev.


In [10]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.


## Lower-Level: Debuggability and Control

Agent Control

The key benefits:
- Decoupling of task creation and execution: Users gain the flexibility to schedule tsk execution according to their needs.
- Enhanced Debuggability: Offers deeper insights into each step of execution process, improving trobubleshooting capabilityies.
- Steerability: Allows users to directly modify intermediate steps and incorporate human feedback for refined control.


In [11]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [12]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [13]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities tailored to different aspects of the software development process, such as conducting business-oriented analysis, translating requirements into system design components, distributing tasks, executing code, formulating test cases for code quality, and more. These roles work together in a structured workflow following Standard Operating Procedures (SOPs) to efficiently develop software solutions.


In [14]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 79fbca88-5a08-4bbe-a92f-de6d52e4f0ec: 1
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities tailored to different aspects of the software development process, such as conducting business-oriented analysis, translating requirements into system design components, distributing tasks, executing code, formulating test cases for code quality, and more. These roles work together in a structured workflow following Standard Operating Procedures (SOPs) to efficiently develop software solutions.


In [15]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 79fbca88-5a08-4bbe-a92f-de6d52e4f0ec: 1


TaskStep(task_id='79fbca88-5a08-4bbe-a92f-de6d52e4f0ec', step_id='262904e2-ac2f-408f-991a-f56d69d26da4', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [16]:
step_output = agent.run_step(task.task_id, input="What about how agents share information?")

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}


Retrying llama_index.llms.openai.base.OpenAI._achat in 0.01644656133321487 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-LXJdh8mBCdaeYl79io2xzzqI on tokens per min (TPM): Limit 80000, Used 76554, Requested 3855. Please try again in 306ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.9151580621582373 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-LXJdh8mBCdaeYl79io2xzzqI on tokens per min (TPM): Limit 80000, Used 76331, Requested 4046. Please try again in 282ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.


=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool for publishing structured messages. They can also subscribe to relevant messages based on their profiles. This approach enhances role communication efficiency and allows agents to obtain directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to exchange information effectively within the multi-agent system. The shared message pool allows all agents to exchange messages directly, publishing structured messages and accessing messages from other entities transparently. This eliminates the need for one-to-one communication and enhances communication efficiency. The subscription mechanism enables agents to select and follow information based on their role profiles, ensuring they receive only task-related information and avoid distractions fr

In [17]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool for publishing structured messages. They can subscribe to relevant messages based on their profiles, enhancing communication efficiency and enabling them to obtain directional information from other roles and public information from the environment. The publish-subscribe mechanism allows agents to exchange information effectively within the multi-agent system, facilitating transparent message exchange and task-related information dissemination.
True


In [18]:
response = agent.finalize_response(task.task_id)

In [19]:
print(str(response))

Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool for publishing structured messages. They can subscribe to relevant messages based on their profiles, enhancing communication efficiency and enabling them to obtain directional information from other roles and public information from the environment. The publish-subscribe mechanism allows agents to exchange information effectively within the multi-agent system, facilitating transparent message exchange and task-related information dissemination.
